---
title: "Datasets"
description: This module contains the useful functions to deal with candidates datasets and individual candidate.
---

In [1]:
#| default_exp datasets
#| export
import polars as pl
import hvplot.polars
import polars.selectors as cs
from loguru import logger
from random import sample
from datetime import timedelta

from pydantic import BaseModel, Field

## Datasets

Foundational Dataset Class

In [2]:
#| export
from discontinuitypy.utils.basic import df2ts
from discontinuitypy.utils.plot import plot_candidate as _plot_candidate
from discontinuitypy.integration import combine_features, calc_combined_features
from discontinuitypy.core.pipeline import ids_finder

08-Feb-24 10:13:42: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/discontinuitypy/lib/python3.10/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/discontinuitypy/lib/python3.10/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


08-Feb-24 10:13:42: UserWarning: pdpipe: nltk import failed. nltk-dependent  pipeline stages will not be loaded.



In [8]:
# | export
class IdsEvents(BaseModel):
    class Config:
        extra = "allow"
        arbitrary_types_allowed = True

    name: str = None
    events: pl.DataFrame = None

    data: pl.LazyFrame = None
    ts: timedelta = None
    """time resolution of the dataset"""
    tau: timedelta = None
    """time interval used to find events"""

    def export(self, path, format="arrow", clean=True, **kwargs):
        if self.events is None:
            self.find_events()
        _df = self.events
        if clean:
            _df = _df.select(cs.datetime(), cs.duration(), cs.numeric())

        match format:
            case "arrow":
                _df.write_ipc(path, **kwargs)
            case "csv":
                _df.write_csv(path, **kwargs)
            case "parquet":
                _df.write_parquet(path, **kwargs)
        return self

    def find_events(self, **kwargs):
        self.events = ids_finder(self.data, ts=self.ts, tau=self.tau, **kwargs)
        return self

    def get_event(self, index=None, predicates=None, random: bool = True, **kwargs):
        if index:
            candidate = self.events.row(index, named=True)
        elif predicates:
            candidate = self.events.filter(predicates).row(0, named=True)
        else:
            index = sample(range(len(self.events)), 1)[0] if random else 0
            candidate = self.events.row(index, named=True)
        return candidate

    def get_event_data(self, candidate=None, index=None, predicates=None, **kwargs):
        if candidate is None:
            candidate = self.get_event(index, predicates, **kwargs)

        _data = self.data.filter(
            pl.col("time").is_between(candidate["tstart"], candidate["tstop"])
        )
        return df2ts(_data, self.bcols)

In [28]:
# | export
class IDsDataset(IdsEvents):

    data: pl.LazyFrame = Field(default=None, alias="mag_data")
    plasma_data: pl.LazyFrame = None

    bcols: list[str] = None
    vec_cols: list[str] = None

    density_col: str = "n"
    speed_col: str = "v"
    temperature_col: str = "T"
    plasma_meta: dict = None

    def update_candidates_with_plasma_data(self, **kwargs):
        df_combined = combine_features(
            self.events,
            self.plasma_data.collect(),
            subset_cols=self.vec_cols
            + [self.density_col, self.speed_col, self.temperature_col],
            **kwargs,
        )

        self.events = calc_combined_features(
            df_combined, vec_cols=self.vec_cols, **kwargs, density_col=self.density_col
        )
        return self

    def plot(self, type="overview", event=None, index=None, predicates=None, **kwargs):

        event = event or self.get_event(index, predicates, **kwargs)
        if type == "overview":
            return self.overview_plot(event, **kwargs)

    def overview_plot(self, event: dict, start=None, stop=None, **kwargs):
        start = start or event["tstart"]
        stop = stop or event["tstop"]

        _plasma_data = self.plasma_data.filter(
            pl.col("time").is_between(start, stop)
        ).collect()

        _mag_data = (
            self.data.filter(pl.col("time").is_between(start, stop))
            .collect()
            .melt(
                id_vars=["time"],
                value_vars=self.bcols,
                variable_name="B comp",
                value_name="B",
            )
        )

        v_df = _plasma_data.melt(
            id_vars=["time"],
            value_vars=self.vec_cols,
            variable_name="veloity comp",
            value_name="v",
        )

        panel_mag = _mag_data.hvplot(
            x="time", y="B", by="B comp", ylabel="Magnetic Field", **kwargs
        )
        panel_n = _plasma_data.hvplot(x="time", y=self.density_col, **kwargs)
        panel_v = v_df.hvplot(
            x="time", y="v", by="veloity comp", ylabel="Plasma Velocity", **kwargs
        )
        panel_temp = _plasma_data.hvplot(x="time", y=self.temperature_col, **kwargs)

        logger.info(f"Overview plot: {event['tstart']} - {event['tstop']}")
        logger.debug(
            f"n.change: {event['n.change']}, $\Delta v_{{ion}}$: {event['v.ion.change']}, T.change: {event['T.change']}, v.Alfven.change: {event['v.Alfven.change']}"
        )

        return (panel_mag + panel_n + panel_v + panel_temp).cols(1)

    def plot_candidate(self, candidate=None, index=None, predicates=None, **kwargs):
        if candidate is None:
            candidate = self.get_event(index, predicates, **kwargs)
        sat_fgm = self.get_event_data(candidate)

        return _plot_candidate(candidate, sat_fgm, **kwargs)

    def plot_candidates(self, indices=None, num=4, random=True, **kwargs):
        if indices is None:
            indices = sample(range(len(self.events)), num) if random else range(num)
            logger.info(f"Candidates indices: {indices}")

        return [self.plot_candidate(index=i, **kwargs) for i in indices]

In [2]:
len({'a':1})

1

In [37]:
#| hide
from nbdev import nbdev_export
nbdev_export()